# Preprocessing for guide-level DE analysis

Test single-guide effects of perturbations used for clustering.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
import numpy as np
import anndata
import pandas as pd
import mudata as md
import scanpy as sc
import glob
import yaml
import argparse

import matplotlib.pyplot as plt
import seaborn as sns

# Add the parent directory to the path to import from sibling directory
sys.path.append(os.path.abspath('../'))
from utils import feature_selection, _convert_oak_path


In [3]:
config = 'DE_config_by_guide.yaml'
# Load configuration from YAML file
with open(config, 'r') as config_file:
    config = yaml.safe_load(config_file)

# Extract parameters from config
datadir = _convert_oak_path(config['datadir'])
experiment_name = config['experiment_name']
datadir = f'{datadir}/{experiment_name}'

# Get parameters with defaults
min_replicates = config.get('min_replicates', 3)
min_cells_per_guide = config.get('min_cells_per_guide', 5)
n_hvgs = config.get('n_hvgs', 10000)
chunk_size = config.get('chunk_size', 50)
chunk_split_seed = config.get('chunk_split_seed', 1423)

# Feature selection parameters
feature_selection_params = config.get('feature_selection', {})
highx_min_mean_counts = feature_selection_params.get('highx_min_mean_counts', 1000)
highx_min_pct_dropouts = feature_selection_params.get('highx_min_pct_dropouts', 0.5)
lowx_max_pct_dropouts = feature_selection_params.get('lowx_max_pct_dropouts', 99.9)
lowx_min_counts = feature_selection_params.get('lowx_min_counts', 99.9)

# File paths
file_paths = config.get('file_paths', {})
pseudobulk_file = f"{experiment_name}_merged_by_guide.DE_pseudobulk.h5ad"
no_effect_guides_file = file_paths.get('no_effect_guides_file', 'no_effect_guides.txt')
de_test_genes_file = file_paths.get('de_test_genes_file', 'DE_test_genes.txt')
target2chunk_file = file_paths.get('target2chunk_file', 'DE_target2chunk.csv.gz')


In [4]:
de_test_genes_file

'DE_test_genes.txt'

## Read pseudobulk objects

Prepared for full DE analysis (`prep_DE.ipynb`)

Read objects

In [5]:
try:
    pbulk_adata
except NameError:
    pbulk_adata = anndata.experimental.read_lazy(f'{datadir}/{experiment_name}_merged.DE_pseudobulk.h5ad')

In [6]:
pbulk_adata.obs = pbulk_adata.obs.to_dataframe()

### Get testable guides

In [7]:
# Combine all conditions into a single keep_for_DE column
pbulk_adata.obs['keep_for_DE'] = (
    pbulk_adata.obs['keep_min_cells'] & 
    pbulk_adata.obs['keep_effective_guides'] & 
    pbulk_adata.obs['keep_total_counts']
)

In [9]:

# Read set of genes used for clustering (by Ron)
DE_stats = pd.read_csv(f'../../metadata/for_DE_by_guide.csv', index_col = 0 )
all_conditions = DE_stats.condition.unique()
DE_stats['target_name_condition'] = DE_stats['target_name'].astype(str) + '_' + DE_stats['condition'].astype(str) 
cond_targets = {}
for c in all_conditions:
    cond_targets[c] = DE_stats[DE_stats.condition == c].target_name.tolist()

In [10]:
for x in cond_targets.values():
    print(len(x))

1605
1518
1498


In [31]:
guide_cell_counts = pbulk_adata.obs[['guide_id', 'perturbed_gene_name', 'culture_condition', 'n_cells', 'keep_for_DE']]
guide_cell_counts = guide_cell_counts[~guide_cell_counts['guide_id'].str.startswith('NTC-')] 
guide_cell_counts['guide_id'] = guide_cell_counts['guide_id'].astype(str)
guide_cell_counts['perturbed_gene_name'] = guide_cell_counts['perturbed_gene_name'].astype(str)
guide_cell_counts['culture_condition'] = guide_cell_counts['culture_condition'].astype(str)

# Count number of replicates per guide in each condition
guides_replicates = guide_cell_counts.groupby(['guide_id', 'perturbed_gene_name', 'culture_condition'])['keep_for_DE'].sum()\
    .sort_values()\
    .reset_index()\
    .pivot(index=['guide_id','perturbed_gene_name'], columns='culture_condition', values='keep_for_DE')\
    .fillna(0)

guides2test = guides_replicates >= min_replicates
guides2test = guides2test.reset_index()
# Filter by min replicates
guides2test = pd.DataFrame([
    {'guide_id': guide_id, 'perturbed_gene_name': perturbed_gene_name, 'condition': condition}
    for condition in all_conditions
    for guide_id, perturbed_gene_name in zip(
        guides2test['guide_id'][
            guides2test[condition] & 
            (guides2test.perturbed_gene_name.isin(cond_targets[condition]))
        ],
        guides2test['perturbed_gene_name'][
            guides2test[condition] & 
            (guides2test.perturbed_gene_name.isin(cond_targets[condition]))
        ]
    )
])
# Test only if both guides are testable for the same perturbation
guide_counts = guides2test.groupby(['condition', 'perturbed_gene_name']).size()
guides2test = guides2test[guides2test.groupby(['condition', 'perturbed_gene_name']).transform('size') == 2]
guides2test_dict = {}
for c in all_conditions:
    guides2test_dict[c] = guides2test[guides2test.condition == c]['guide_id'].values.tolist()

for c in guides2test_dict.keys():
    print(c, ':', len(guides2test_dict[c]))

Stim8hr : 2350
Stim48hr : 2118
Rest : 2040


## Filter samples for DE analysis

In [32]:
# Keep genes that pass filters for each condition separately
pbulk_adata.obs['keep_test_genes'] = False  # Initialize as all False

# For each condition, mark genes that pass filters for that condition
for condition, genes in guides2test_dict.items():
    mask = (pbulk_adata.obs['culture_condition'] == condition) & \
           (pbulk_adata.obs['guide_id'].isin(genes))
    pbulk_adata.obs.loc[mask, 'keep_test_genes'] = True

# Always keep NTC samples
pbulk_adata.obs.loc[pbulk_adata.obs['perturbed_gene_name'] == 'NTC', 'keep_test_genes'] = True

# Aggregate with the existing keep_for_DE condition
pbulk_adata.obs['keep_for_DE'] = pbulk_adata.obs['keep_for_DE'] & pbulk_adata.obs['keep_test_genes']

pbulk_adata.obs['keep_for_DE'].value_counts()

keep_for_DE
False    244448
True      34255
Name: count, dtype: int64

In [33]:
pbulk_adata = pbulk_adata[pbulk_adata.obs['keep_for_DE']].to_memory()

In [34]:
pbulk_adata

AnnData object with n_obs × n_vars = 34255 × 18129
    obs: '10xrun_id', 'cell_sample_id', 'donor_id', 'culture_condition', 'guide_id', 'perturbed_gene_name', 'perturbed_gene_id', 'guide_type', 'n_cells', 'total_counts', 'keep_min_cells', 'keep_effective_guides', 'keep_total_counts', 'keep_for_DE', 'keep_test_genes', 'log10_n_cells'
    var: 'gene_ids', 'gene_name'

In [35]:
pbulk_adata.write_h5ad(f'{datadir}/{pseudobulk_file}')

In [36]:
try:
    pbulk_adata
except NameError:
    pbulk_adata = anndata.experimental.read_lazy(f'{datadir}/{pseudobulk_file}')
    pbulk_adata = pbulk_adata[pbulk_adata.obs['keep_for_DE']].copy()
    pbulk_adata.obs = pbulk_adata.obs.to_dataframe()

In [37]:
pbulk_adata

AnnData object with n_obs × n_vars = 34255 × 18129
    obs: '10xrun_id', 'cell_sample_id', 'donor_id', 'culture_condition', 'guide_id', 'perturbed_gene_name', 'perturbed_gene_id', 'guide_type', 'n_cells', 'total_counts', 'keep_min_cells', 'keep_effective_guides', 'keep_total_counts', 'keep_for_DE', 'keep_test_genes', 'log10_n_cells'
    var: 'gene_ids', 'gene_name'

## Feature selection

In [6]:
### USE SAME GENES USED FOR FULL TEST ###

# feature_selection_vars = pd.DataFrame()
# for c in pbulk_adata.obs['culture_condition'].unique():
#     print(c)
#     pbulk_adata_cond = pbulk_adata[pbulk_adata.obs['culture_condition'] == c].copy()
#     feature_selection_var = feature_selection(
#         pbulk_adata_cond,
#         n_hvgs = n_hvgs,
#         subset_adata=False,
#         highx_min_mean_counts = highx_min_mean_counts,
#         highx_min_pct_dropouts_by_counts = highx_min_pct_dropouts,
#         lowx_max_pct_dropouts_by_counts = lowx_max_pct_dropouts,
#         lowx_min_counts = lowx_min_counts,
#         return_all = True,
#         use_rapids = False
#         )
    
#     # Remove PuroR gene if it exists in the feature selection results
#     if 'CUSTOM001_PuroR' in feature_selection_var.index:
#         feature_selection_var = feature_selection_var.drop('CUSTOM001_PuroR')

#     all_targets = pbulk_adata_cond.obs['perturbed_gene_id'].unique()
#     feature_selection_var['is_target'] = feature_selection_var.index.isin(all_targets)

#     DE_test_genes = feature_selection_var[feature_selection_var['highly_variable'] | feature_selection_var['is_target']].index.tolist()
#     # Save DE_test_genes to a text file
#     with open(f"{datadir}/{de_test_genes_file.replace('.txt', f'.{c}.txt')}", 'w') as f:
#         for gene in DE_test_genes:
#             f.write(f"{gene}\n")
#     feature_selection_var['culture_condition'] = c
#     feature_selection_vars = pd.concat([feature_selection_vars, feature_selection_var])

In [ ]:
## --- SPLIT PERTURBATIONS INTO CHUNKS --- ##
for c in pbulk_adata.obs['culture_condition'].unique():
    print(c)
    pbulk_adata_cond = pbulk_adata[pbulk_adata.obs['culture_condition'] == c].copy()
    all_targets = pbulk_adata_cond.obs['guide_id'].unique().tolist()
    all_targets = [x for x in all_targets if not x.startswith('NTC')] # exclude NTCs

    # Randomize targets before splitting (without replacement)
    np.random.seed(chunk_split_seed)
    np.random.shuffle(all_targets)

    # Split all_targets into groups based on chunk_size
    target_chunks = [all_targets[i:i+chunk_size] for i in range(0, len(all_targets), chunk_size)]

    # Initialize a binary matrix with zeros
    target_chunk_matrix = pd.DataFrame(0, 
                                    index=all_targets, 
                                    columns=[f'chunk_{i}' for i in range(len(target_chunks))])

    # Fill the matrix with 1s for each target in its respective chunk
    for chunk_idx, chunk in enumerate(target_chunks):
        
        target_chunk_matrix.loc[chunk, f'chunk_{chunk_idx}'] = 1

    target_chunk_matrix.to_csv(f"{datadir}/{target2chunk_file.replace('.csv.gz', f'.{c}.csv.gz')}", compression='gzip')

Stim48hr
Stim8hr
Rest


In [39]:
target_chunk_matrix.head()

,chunk_0,chunk_1,chunk_2,chunk_3,chunk_4,chunk_5,chunk_6,chunk_7,chunk_8,chunk_9,...,chunk_31,chunk_32,chunk_33,chunk_34,chunk_35,chunk_36,chunk_37,chunk_38,chunk_39,chunk_40
SHLD1-2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PMM2-1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ISG15-1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GMEB1-1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
KRBA2-1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

In [43]:
# Annotate number of cells per target gene
all_conditions = ['Rest', 'Stim48hr', 'Stim8hr']
guide_cell_counts = pd.DataFrame()
for culture_condition in all_conditions:
    guide_cell_counts_c = pd.read_csv(f'{datadir}/{experiment_name}.guide_effect.{culture_condition}.csv', index_col=0)[['guide_n']]
    guide_cell_counts_c['culture_condition'] = culture_condition
    guide_cell_counts_c['perturbed_gene_id'] = sgrna_library_metadata.set_index('sgrna_id').loc[guide_cell_counts_c.index]['perturbed_gene_id']
    guide_cell_counts = pd.concat([guide_cell_counts, guide_cell_counts_c])
n_cells_target_contrast = guide_cell_counts.groupby(['perturbed_gene_id', 'culture_condition'])['guide_n'].sum().reset_index()
n_cells_target_contrast.index = n_cells_target_contrast['perturbed_gene_id'] + '_' + n_cells_target_contrast['culture_condition']

all_tested_conds = []
for culture_condition in all_conditions:
    cond_cond = pd.read_csv(f'{datadir}/DE_target2chunk.{culture_condition}.csv.gz', index_col=0).index + '_' + culture_condition
    all_tested_conds.extend(cond_cond.tolist())

In [58]:
common = np.intersect1d(all_tested_conds, n_cells_target_contrast.index)
pd.Series(all_tested_conds)[~pd.Series(all_tested_conds).isin(common)]

91           ENSG00000160959_Rest
1347         ENSG00000070882_Rest
1856         ENSG00000159593_Rest
2367         ENSG00000102678_Rest
3031         ENSG00000090372_Rest
3315         ENSG00000198231_Rest
5159         ENSG00000104957_Rest
6987         ENSG00000007202_Rest
8089         ENSG00000126003_Rest
8267         ENSG00000106803_Rest
9002         ENSG00000127080_Rest
9902         ENSG00000136045_Rest
9966         ENSG00000085998_Rest
10049        ENSG00000112658_Rest
11130        ENSG00000168538_Rest
15885    ENSG00000167258_Stim48hr
21345    ENSG00000129691_Stim48hr
24846     ENSG00000069248_Stim8hr
30634     ENSG00000141012_Stim8hr
31411     ENSG00000155265_Stim8hr
32987     ENSG00000129691_Stim8hr
dtype: object

In [62]:
culture_condition = 'Rest'
cond_cond = pd.read_csv(f'{datadir}/DE_target2chunk.{culture_condition}.csv.gz', index_col=0)

In [ ]:
cond_cond

,chunk_0,chunk_1,chunk_2,chunk_3,chunk_4,chunk_5,chunk_6,chunk_7,chunk_8,chunk_9,...,chunk_216,chunk_217,chunk_218,chunk_219,chunk_220,chunk_221,chunk_222,chunk_223,chunk_224,chunk_225
ENSG00000121390,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000116922,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000130733,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000145246,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000163600,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000006015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ENSG00000196352,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ENSG00000103037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ENSG00000198408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [65]:
de_results_dir = datadir + f'/DE_results_all_confounders/tmp/'
file = de_results_dir + 'DE_results.Rest.chunk_1.csv.gz'
df = pd.read_csv(file, compression='gzip', index_col=0)

In [67]:
df[df['contrast'] == 'ENSG00000160959']

,variable,baseMean,log_fc,lfcSE,stat,p_value,adj_p_value,contrast,culture_condition
605934,ENSG00000147443,261.839502,0.654228,0.134081,4.879367,0.000001,0.013160,ENSG00000160959,Rest
605935,ENSG00000184009,1012.094899,0.519047,0.117668,4.411115,0.000010,0.049336,ENSG00000160959,Rest
605936,ENSG00000153560,75.255453,0.680497,0.155431,4.378134,0.000012,0.049336,ENSG00000160959,Rest
605937,ENSG00000136051,81.851833,-1.079410,0.269153,-4.010396,0.000061,0.170533,ENSG00000160959,Rest
605938,ENSG00000102572,227.611136,0.431009,0.108298,3.979852,0.000069,0.170533,ENSG00000160959,Rest
...,...,...,...,...,...,...,...,...,...
618295,ENSG00000170293,4.136770,0.000080,0.879847,0.000091,0.999928,0.999973,ENSG00000160959,Rest
618296,ENSG00000051382,11.814585,-0.000042,0.481128,-0.000086,0.999931,0.999973,ENSG00000160959,Rest
618297,ENSG00000165474,1.269691,-0.000195,2.658116,-0.000073,0.999942,0.999973,ENSG00000160959,Rest
618298,ENSG00000167768,0.052719,-0.000222,6.473701,-0.000034,0.999973,0.999973,ENSG00000160959,Rest


In [80]:
pbulk_adata_obs = anndata.experimental.read_lazy(f'{datadir}/{experiment_name}_merged.DE_pseudobulk.h5ad').obs.to_dataframe()
n_cells_target_contrast = pbulk_adata_obs[['perturbed_gene_id', 'culture_condition', 'n_cells']].copy()
n_cells_target_contrast['test'] = n_cells_target_contrast['perturbed_gene_id'].astype(str) + '_' + n_cells_target_contrast['culture_condition'].astype(str)
n_cells_target_contrast = n_cells_target_contrast.groupby('test')['n_cells'].sum().reset_index().set_index('test')

In [83]:
n_cells_target_contrast

,n_cells
test,
AKAP2_Rest,1188.0
AKAP2_Stim48hr,1187.0
AKAP2_Stim8hr,1174.0
ENSG00000000003_Rest,686.0
ENSG00000000003_Stim48hr,713.0
...,...
SMIM11B_Stim48hr,1978.0
SMIM11B_Stim8hr,1854.0
U2AF1L5_Rest,88.0


Testing DE analysis

In [7]:
# Extract parameters from config
datadir = _convert_oak_path(config['datadir'])
experiment_name = config['experiment_name']
run_name = config.get('run_name', 'default')

datadir = f'{datadir}/{experiment_name}'
chunk_ix = 1
cond = 'Stim8hr'

# Create directories for results
de_results_dir = f'{datadir}/DE_results_{run_name}/'
de_results_tmp_dir = f'{de_results_dir}/tmp/'
os.makedirs(de_results_dir, exist_ok=True)
os.makedirs(de_results_tmp_dir, exist_ok=True)

pbulk_adata = sc.read_h5ad(f'{datadir}/{experiment_name}_merged.DE_pseudobulk.h5ad')
pbulk_adata = pbulk_adata[(pbulk_adata.obs['keep_for_DE']) & (pbulk_adata.obs['culture_condition'] == cond)].copy()


In [11]:
config = 'DE_config_full.yaml'
with open(config, 'r') as config_file:
    config = yaml.safe_load(config_file)

In [13]:
with open(f'{datadir}/DE_single_guide_test_genes.{cond}.txt', 'r') as f:
    de_test_genes = [line.strip() for line in f if line.strip()]

In [14]:
sys.path.append(os.path.abspath('../'))
from utils import _convert_oak_path

from copy import deepcopy
import argparse

from MultiStatePerturbSeqDataset import *

/home/emmadann/miniforge3/envs/rpy2-voodoo/lib/python3.12/site-packages/anndata/utils.py:435: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/emmadann/miniforge3/envs/rpy2-voodoo/lib/python3.12/site-packages/anndata/utils.py:435: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/emmadann/miniforge3/envs/rpy2-voodoo/lib/python3.12/site-packages/anndata/utils.py:435: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/emmadann/miniforge3/envs/rpy2-voodoo/lib/python3.12/site-packages/anndata/utils.py:435: FutureWarning: Importing CSCDataset from `anndata.experimental` is deprecated. Import anndata.abc.CSCDataset instead.
  warnings.warn(msg, FutureWarning)
/home/emmadann/miniforge3/envs/rpy2-voodoo/lib/python3.12/

In [16]:
pbulk_adata.obs['guide_test'] = np.where(pbulk_adata.obs['guide_id'].str.startswith('NTC'), 'NTC', pbulk_adata.obs['guide_id'])
pbulk_adata = pbulk_adata[:, de_test_genes].copy()

# Get targets to test
target_chunk_matrix = pd.read_csv(f'{datadir}/DE_single_guide_target2chunk.{cond}.csv.gz', compression='gzip', index_col=0)
test_targets = target_chunk_matrix.index[target_chunk_matrix[f'chunk_{chunk_ix}'] == 1].tolist()

# Run DE analysis
ms_perturb_data = MultistatePerturbSeqDataset(
    pbulk_adata,
    sample_cols = ['cell_sample_id'],
    perturbation_type = 'CRISPRi',
    target_col = 'guide_test',
    sgrna_col = 'guide_id',
    state_col = 'culture_condition',
    control_level = 'NTC'
    )


In [18]:
st = cond
# Get DE parameters from config
run_de_params = config.get('run_DE_params', {})
design_formula = run_de_params.get('design_formula', '~ log10_n_cells + target')
min_counts_per_gene = run_de_params.get('min_counts_per_gene', 10)

adata = ms_perturb_data.adata
adata_state = adata[(adata.obs[ms_perturb_data.cell_state_obs] == st) & (adata.obs['target'].isin(test_targets + [ms_perturb_data.control_level]))]
adata_state = adata_state[:, adata_state.X.sum(0) >= min_counts_per_gene].copy()

In [23]:
min_counts_per_gene

10

In [24]:
adata_state = sc.read_h5ad('pbulk_test.h5ad')
design_formula = '~ log10_n_cells + donor_id + target'
min_counts_per_gene = 10
n_cpus = 3

model = pertpy.tl.PyDESeq2(adata_state, design=design_formula)
model.fit(n_cpus = n_cpus, quiet=True)

all_targets = adata_state.obs['target'].unique().tolist()
all_targets.remove(self.control_level)

for t in tqdm(all_targets, desc="Testing targets"):
        t_contrast = (model.cond(target = t) - model.cond(target = self.control_level)) 
        res_df = model.test_contrasts(t_contrast)
        res_df[self.cell_state_obs] = st
        res_df['contrast'] = t
        all_res_df = pd.concat([all_res_df, res_df])

        all_res_df = all_res_df.reset_index().drop('index', axis=1)


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


KeyboardInterrupt: 

In [ ]:
pbulk_adata = anndata.experimental.read_lazy(f'{datadir}/{experiment_name}_merged_by_guide.DE_pseudobulk.h5ad')
pbulk_adata = pbulk_adata[(pbulk_adata.obs['keep_for_DE']) & (pbulk_adata.obs.culture_condition == cond)].to_memory()


In [43]:
tets_ad = pbulk_adata[:, pbulk_adata.var_names[0]].to_memory()

tets_ad

AnnData object with n_obs × n_vars = 278703 × 1
    obs: '10xrun_id', 'cell_sample_id', 'donor_id', 'culture_condition', 'guide_id', 'perturbed_gene_name', 'perturbed_gene_id', 'guide_type', 'n_cells', 'total_counts', 'keep_min_cells', 'keep_effective_guides', 'keep_total_counts', 'keep_for_DE', 'keep_test_genes', 'log10_n_cells'
    var: 'gene_ids', 'gene_name'

AnnData object with n_obs × n_vars = 278703 × 1
    obs: '10xrun_id', 'cell_sample_id', 'donor_id', 'culture_condition', 'guide_id', 'perturbed_gene_name', 'perturbed_gene_id', 'guide_type', 'n_cells', 'total_counts', 'keep_min_cells', 'keep_effective_guides', 'keep_total_counts', 'keep_for_DE', 'keep_test_genes', 'log10_n_cells'
    var: 'gene_ids', 'gene_name'

In [77]:
all_chunk_outs = glob.glob('/mnt/oak/users/emma/data/GWT//CD4iR1_Psomagen/DE_results_all_confounders/tmp/DE_results.Rest*.csv.gz')
all_res = []
for f in all_chunk_outs:
    res = pd.read_csv(f, compression='gzip')
    if any(res.variable == 'ENSG00000111537'):
        continue
    else:
        print('missing IFNG')
    all_res.append(res)

In [27]:
sgrna_library_metadata = pd.read_csv('../../metadata/sgRNA_library_curated.csv', index_col=0)
gene_name_to_id = dict(zip(sgrna_library_metadata['perturbed_gene_id'], sgrna_library_metadata['perturbed_gene_name']))
# var_df = sc.read_h5ad(f'{datadir}/{experiment_name}_merged.DE_pseudobulk.h5ad', backed=True).var.copy()

In [101]:
from merge_DE_results import parse_DE_results_2_adata
from tqdm import tqdm
de_results_dir = '/mnt/oak/users/emma/data/GWT//CD4iR1_Psomagen/DE_results_all_confounders/tmp/'
# Read all csv.gz files from the DE results directory
de_results_files = glob.glob(de_results_dir + 'DE_results.*.csv.gz')
de_results_adatas = []

for file in tqdm(de_results_files, desc="Processing DE result files"):
    try:
        df = pd.read_csv(file, compression='gzip', index_col=0)
        df = df.rename({'contrast': 'target_contrast'}, axis=1)
        df['target_contrast_gene_name'] = df['target_contrast'].map(lambda x: gene_name_to_id.get(x, x))
        de_results_adatas.append(parse_DE_results_2_adata(df))
    except EOFError:
        continue


Processing DE result files: 100%|██████████| 351/351 [07:53<00:00,  1.35s/it]


In [117]:
all_dfs = {}
for stat in ['baseMean', 'log_fc', 'lfcSE', 'p_value','adj_p_value']:
    # Use all_variables to ensure consistent columns across stats
    stat_df = df.pivot(values=stat, columns='variable', index='target_contrast')
    # Reindex to include all variables, filling missing values with NaN
    # stat_df = stat_df.reindex(columns=all_variables)
    all_dfs[stat] = stat_df

DE_anndata = anndata.AnnData(
        layers = all_dfs.copy()
    )

In [123]:
np.isnan(DE_anndata.layers['adj_p_value'])

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [130]:
file = de_results_files[0]
df = pd.read_csv(file, compression='gzip', index_col=0)
df = df.rename({'contrast': 'target_contrast'}, axis=1)

In [136]:
file

'/mnt/oak/users/emma/data/GWT//CD4iR1_Psomagen/DE_results_all_confounders/tmp/DE_results.Stim8hr.chunk_178.csv.gz'

In [129]:
de_results_adatas[6][de_results_adatas[6].var_names == 'ENSG00000111537']

IndexError: Boolean index does not match AnnData’s shape along this dimension. Boolean index has shape (10981,) while AnnData index has shape (50,).

In [124]:
['ENSG00000111537' in x.var_names for x in de_results_adatas]

[False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 T

In [92]:
a = parse_DE_results_2_adata(df)

In [100]:
a[:, a.var_names == 'ENSG00000111537']

View of AnnData object with n_obs × n_vars = 50 × 1
    obs: 'target_contrast_gene_name', 'culture_condition', 'target_contrast'
    layers: 'baseMean', 'log_fc', 'lfcSE', 'p_value', 'adj_p_value'

In [33]:
cond = 'Rest'
chunk_ix = 1

In [ ]:


# # Create directories for results
# de_results_dir = f'{datadir}/DE_results_{run_name}/'
# de_results_tmp_dir = f'{de_results_dir}/tmp/'
# os.makedirs(de_results_dir, exist_ok=True)
# os.makedirs(de_results_tmp_dir, exist_ok=True)

# pbulk_adata = sc.read_h5ad(f'{datadir}/{experiment_name}_merged.DE_pseudobulk.h5ad')
# pbulk_adata = pbulk_adata[pbulk_adata.obs['keep_for_DE']].copy()

# Read list of genes for DE testing
try:
    with open(f'{datadir}/DE_test_genes.{cond}.txt', 'r') as f:
        de_test_genes = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(de_test_genes)} genes for DE testing")
except FileNotFoundError:
    raise(FileNotFoundError, f"Warning: DE test genes file not found at {datadir}/DE_test_genes.txt - run feature selection first.")

pbulk_adata = pbulk_adata[:, de_test_genes].copy()

# Get targets to test
target_chunk_matrix = pd.read_csv(f'{datadir}/DE_target2chunk.{cond}.csv.gz', compression='gzip', index_col=0)
test_targets = target_chunk_matrix.index[target_chunk_matrix[f'chunk_{chunk_ix}'] == 1].tolist()

Loaded 13076 genes for DE testing


In [37]:
test_targets

50

In [38]:
run_de_params = config.get('run_DE_params', {})
design_formula = run_de_params.get('design_formula', '~ log10_n_cells + target')
min_counts_per_gene = run_de_params.get('min_counts_per_gene', 10)

model, results = ms_perturb_data.run_target_DE(
    design_formula = design_formula,
    test_state = [cond], test_targets=test_targets,
    min_counts_per_gene = min_counts_per_gene,
    return_model = True
    )

Fitting dispersions...
... done in 0.02 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.02 seconds.

Testing targets:   0%|          | 0/50 [00:00<?, ?it/s]Running Wald tests...
... done in 1.77 seconds.

Testing targets:   2%|▏         | 1/50 [00:02<01:46,  2.17s/it]

Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  441.499348       -0.447468   0.230146 -1.944276  0.051862   
ENSG00000000003    0.002106       -0.003517  32.478580 -0.000108  0.999914   
ENSG00000000419   67.331358        0.046903   0.096986  0.483601  0.628669   
ENSG00000000457   18.053288       -0.201637   0.172709 -1.167499  0.243009   
ENSG00000000460   40.371277       -0.332326   0.208248 -1.595814  0.110530   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.214440        0.004138   1.349926  0.003065  0.997554   
ENSG00000291110    0.103885       -0.414982   1.859145 -0.223211  0.823371   
ENSG00000291122    4.988385

Running Wald tests...
... done in 2.03 seconds.

Testing targets:   4%|▍         | 2/50 [00:04<01:51,  2.31s/it]

Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  441.499348       -0.360641   0.229469 -1.571632  0.116036   
ENSG00000000003    0.002106       -0.226642  32.487824 -0.006976  0.994434   
ENSG00000000419   67.331358        0.017078   0.112602  0.151670  0.879447   
ENSG00000000457   18.053288        0.147284   0.182530  0.806906  0.419721   
ENSG00000000460   40.371277        0.199420   0.211569  0.942577  0.345897   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.214440       -0.661925   1.611561 -0.410735  0.681267   
ENSG00000291110    0.103885       -0.460804   1.852173 -0.248791  0.803522   
ENSG00000291122    4.988385

Running Wald tests...
... done in 2.06 seconds.

Testing targets:   6%|▌         | 3/50 [00:07<01:51,  2.38s/it]

...

Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  441.499348       -0.413303   0.224409 -1.841739  0.065513   
ENSG00000000003    0.002106       -0.081865  32.477412 -0.002521  0.997989   
ENSG00000000419   67.331358       -0.129179   0.086413 -1.494900  0.134941   
ENSG00000000457   18.053288        0.074628   0.129527  0.576159  0.564508   
ENSG00000000460   40.371277       -0.017075   0.184213 -0.092690  0.926150   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.214440        0.459621   0.969373  0.474143  0.635398   
ENSG00000291110    0.103885        1.164035   1.184605  0.982636  0.325787   
ENSG00000291122    4.988385

Running Wald tests...
... done in 2.30 seconds.

Testing targets:  98%|█████████▊| 49/50 [02:00<00:02,  2.61s/it]

Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  441.499348        0.025580   0.255797  0.100000  0.920345   
ENSG00000000003    0.002106       -0.009682  37.491523 -0.000258  0.999794   
ENSG00000000419   67.331358       -0.238864   0.102607 -2.327961  0.019914   
ENSG00000000457   18.053288       -0.050167   0.156951 -0.319637  0.749244   
ENSG00000000460   40.371277        0.060336   0.205486  0.293624  0.769046   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.214440       -1.474425   1.862869 -0.791481  0.428664   
ENSG00000291110    0.103885       -0.814407   2.131569 -0.382069  0.702410   
ENSG00000291122    4.988385

Running Wald tests...
... done in 1.80 seconds.

Testing targets: 100%|██████████| 50/50 [02:02<00:00,  2.45s/it]


Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  441.499348       -0.295047   0.222943 -1.323419  0.185696   
ENSG00000000003    0.002106       -0.001906  32.491862 -0.000059  0.999953   
ENSG00000000419   67.331358       -0.114422   0.074188 -1.542324  0.122995   
ENSG00000000457   18.053288       -0.098451   0.104418 -0.942859  0.345753   
ENSG00000000460   40.371277       -0.112865   0.176923 -0.637930  0.523519   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.214440        1.403050   0.738525  1.899799  0.057459   
ENSG00000291110    0.103885       -0.175945   1.519094 -0.115822  0.907793   
ENSG00000291122    4.988385

,variable,baseMean,log_fc,lfcSE,stat,p_value,adj_p_value,contrast,culture_condition
0,ENSG00000111537,9.931768,1.226809,0.252524,4.858189,1.184643e-06,1.671413e-02,ENSG00000105127,Rest
14111,ENSG00000165240,22.336011,-3.682364,0.596340,-6.174939,6.618920e-10,9.338634e-06,ENSG00000165240,Rest
28222,ENSG00000066427,20.424472,-2.657265,0.270010,-9.841370,7.468688e-23,1.053757e-18,ENSG00000066427,Rest
42333,ENSG00000012822,14.032496,-3.070687,0.244001,-12.584733,2.562063e-36,3.614815e-32,ENSG00000012822,Rest
42334,ENSG00000100888,32.551015,-0.298751,0.063018,-4.740750,2.129286e-06,1.502105e-02,ENSG00000012822,Rest
...,...,...,...,...,...,...,...,...,...
649109,ENSG00000173218,26.587957,-1.919939,0.438572,-4.377703,1.199367e-05,4.230466e-02,ENSG00000102871,Rest
663217,ENSG00000196670,16.219548,-3.655832,0.463382,-7.889460,3.034963e-15,4.282029e-11,ENSG00000196670,Rest
663218,ENSG00000143774,56.077149,-0.425977,0.094840,-4.491513,7.071889e-06,4.988864e-02,ENSG00000196670,Rest
677328,ENSG00000132950,15.489564,-1.366811,0.261045,-5.235912,1.641722e-07,2.316306e-03,ENSG00000132950,Rest
